# Calculating potential market revenue

In [329]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

### Import dataset

In [330]:
price_df = pd.read_csv('../data/clean_data/wholesale_price_2022.csv', index_col=0)
price_df = price_df[['date', 'datetime', 'hour', 'gb_price_eur', 'PUN', 'NORD', 'CNOR', 'CSUD', 'SUD', 'SARD', 'SICI']]
price_df.head()

,date,datetime,hour,gb_price_eur,PUN,NORD,CNOR,CSUD,SUD,SARD,SICI
0,2022-01-01,2022-01-01 00:00:00,1,89.24,170.28,170.28,170.28,170.28,170.28,170.28,170.28
1,2022-01-01,2022-01-01 01:00:00,2,89.24,155.72,155.72,155.72,155.72,155.72,155.72,155.72
2,2022-01-01,2022-01-01 02:00:00,3,108.52,147.09,147.09,147.09,147.09,147.09,147.09,147.09
3,2022-01-01,2022-01-01 03:00:00,4,57.12,91.00,91.00,91.00,91.00,91.00,91.00,91.00
4,2022-01-01,2022-01-01 04:00:00,5,83.29,104.00,104.00,104.00,104.00,104.00,104.00,104.00


### Daily spread

In [332]:
daily_max = price_df.groupby('date').max().reset_index()
daily_min = price_df.groupby('date').min().reset_index()
daily_max.head()

,date,datetime,hour,gb_price_eur,PUN,NORD,CNOR,CSUD,SUD,SARD,SICI
0,2022-01-01,2022-01-01 23:00:00,24,108.52,214.66000,214.66,214.66,214.66,214.66,214.66,214.66
1,2022-01-02,2022-01-02 23:00:00,24,251.07,245.00000,245.00,245.00,245.00,245.00,245.00,245.00
2,2022-01-03,2022-01-03 23:00:00,24,328.04,305.27316,300.00,300.00,300.00,300.00,300.00,382.14
3,2022-01-04,2022-01-04 23:00:00,24,385.11,202.74775,201.21,201.21,201.21,201.21,201.21,285.00
4,2022-01-05,2022-01-05 23:00:00,24,167.27,265.00000,265.00,265.00,265.00,265.00,265.00,265.00


In [253]:
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('GB', 'Italy'),
    shared_xaxes=True
)

fig.add_trace(go.Scatter(
    y=daily_min.gb_price_eur,
    x=daily_min.datetime,
    line=dict(color='crimson', width=2),
    name='GB min'
), row=1, col=1)
fig.add_trace(go.Scatter(
    y=daily_max.gb_price_eur,
    x=daily_max.datetime,
    line=dict(color='green', width=2),
    name='GB max'
), row=1, col=1)
fig.add_trace(go.Scatter(
    y=daily_min.PUN,
    x=daily_min.datetime,
    line=dict(color='crimson', width=2),
    name='Italy PUN min'
), row=2, col=1)
fig.add_trace(go.Scatter(
    y=daily_max.PUN,
    x=daily_max.datetime,
    line=dict(color='green', width=2),
    name='Italy PUN max'
), row=2, col=1)

fig.update_layout(
    title='Daily highs and lows for British and Italian Day-Ahead markets',
    template='ggplot2',
    width=1200,
    height=500,
)
fig.update_yaxes(title='€/MW')

fig.write_image('../figures/gb-italy-da.jpeg', scale=5, engine='orca')
fig

### Single cycle 1 hour

Buy at PUN min sell at zonal max each day

In [333]:
single_cycle_1hr_dict = {
    'NORD': (daily_max.NORD - daily_min.PUN).sum(),
    'CNOR': (daily_max.CNOR - daily_min.PUN).sum(),
    'CSUD': (daily_max.CSUD - daily_min.PUN).sum(),
    'SUD': (daily_max.SUD - daily_min.PUN).sum(),
    'SICI': (daily_max.SICI - daily_min.PUN).sum(),
    'SARD': (daily_max.SARD - daily_min.PUN).sum(),
}

single_cycle_1hr_gb = (daily_max.gb_price_eur - daily_min.gb_price_eur).sum()
print(f'GB revenue: {single_cycle_1hr_gb}')
single_cycle_1hr_df = pd.DataFrame(single_cycle_1hr_dict, index=['revenue']).transpose().round(2)
single_cycle_1hr_df

GB revenue: 50551.87


,revenue
NORD,59735.26
CNOR,59766.75
CSUD,58052.87
SUD,57291.98
SICI,58018.73
SARD,57732.57


In [255]:
fig = px.bar(
    single_cycle_1hr_df,
    color=single_cycle_1hr_df.revenue.values,
    color_continuous_scale='solar',
    color_continuous_midpoint=59e3,
    text_auto='.3s'
)

fig.update_layout(
    title='Single cycle 1 hour revenue',
    xaxis={'categoryorder':'total descending'},
    xaxis_title='',
    yaxis_title='€/MW',
    yaxis_range=[56e3, 60e3],
    template='ggplot2',
    width=1200,
    height=500,
)
fig.update_coloraxes(showscale=False)
fig.update_traces(textfont_size=14, textposition='inside')

fig.write_image('../figures/single-cycle-1hr.jpeg', scale=5, engine='orca')
fig

### Single cycle 2 hour

In [336]:
price_df.head(24)

,date,datetime,hour,gb_price_eur,PUN,NORD,CNOR,CSUD,SUD,SARD,SICI
0,2022-01-01,2022-01-01 00:00:00,1,89.24,170.28,170.28,170.28,170.28,170.28,170.28,170.28
1,2022-01-01,2022-01-01 01:00:00,2,89.24,155.72,155.72,155.72,155.72,155.72,155.72,155.72
2,2022-01-01,2022-01-01 02:00:00,3,108.52,147.09,147.09,147.09,147.09,147.09,147.09,147.09
3,2022-01-01,2022-01-01 03:00:00,4,57.12,91.00,91.00,91.00,91.00,91.00,91.00,91.00
4,2022-01-01,2022-01-01 04:00:00,5,83.29,104.00,104.00,104.00,104.00,104.00,104.00,104.00
5,2022-01-01,2022-01-01 05:00:00,6,0.00,140.60,140.60,140.60,140.60,140.60,140.60,140.60
6,2022-01-01,2022-01-01 06:00:00,7,64.49,147.09,147.09,147.09,147.09,147.09,147.09,147.09
7,2022-01-01,2022-01-01 07:00:00,8,-22.85,99.99,99.99,99.99,99.99,99.99,99.99,99.99
8,2022-01-01,2022-01-01 08:00:00,9,35.70,67.99,67.99,67.99,67.99,67.99,67.99,67.99
9,2022-01-01,2022-01-01 09:00:00,10,-10.23,95.00,95.00,95.00,95.00,95.00,95.00,95.00


In [256]:
pun_min = price_df.groupby('date')['PUN'].nsmallest(2).reset_index()[['date', 'PUN']].groupby('date').sum().reset_index()
nord_max = price_df.groupby('date')['NORD'].nlargest(2).reset_index()[['date', 'NORD']].groupby('date').sum().reset_index()
cnor_max = price_df.groupby('date')['CNOR'].nlargest(2).reset_index()[['date', 'CNOR']].groupby('date').sum().reset_index()
csud_max = price_df.groupby('date')['CSUD'].nlargest(2).reset_index()[['date', 'CSUD']].groupby('date').sum().reset_index()
sud_max = price_df.groupby('date')['SUD'].nlargest(2).reset_index()[['date', 'SUD']].groupby('date').sum().reset_index()
sard_max = price_df.groupby('date')['SARD'].nlargest(2).reset_index()[['date', 'SARD']].groupby('date').sum().reset_index()
sici_max = price_df.groupby('date')['SICI'].nlargest(2).reset_index()[['date', 'SICI']].groupby('date').sum().reset_index()
gb_min = price_df.groupby('date')['gb_price_eur'].nsmallest(2).reset_index()[['date', 'gb_price_eur']].groupby('date').sum().reset_index().rename({'gb_price_eur': 'gb_min'}, axis=1)
gb_max = price_df.groupby('date')['gb_price_eur'].nlargest(2).reset_index()[['date', 'gb_price_eur']].groupby('date').sum().reset_index().rename({'gb_price_eur': 'gb_max'}, axis=1)

two_hour_df = pd.concat([pun_min, nord_max, cnor_max, csud_max, sud_max, sard_max, sici_max, gb_min, gb_max], axis=1)
two_hour_df.head()

,date,PUN,date,NORD,date,CNOR,date,CSUD,date,SUD,date,SARD,date,SICI,date,gb_min,date,gb_max
0,2022-01-01,158.99000,2022-01-01,429.32,2022-01-01,429.32,2022-01-01,429.32,2022-01-01,429.32,2022-01-01,429.32,2022-01-01,429.32,2022-01-01,-49.38,2022-01-01,216.56
1,2022-01-02,240.60000,2022-01-02,470.19,2022-01-02,470.19,2022-01-02,470.19,2022-01-02,470.19,2022-01-02,470.19,2022-01-02,485.00,2022-01-02,296.94,2022-01-02,499.76
2,2022-01-03,167.77000,2022-01-03,600.00,2022-01-03,600.00,2022-01-03,600.00,2022-01-03,600.00,2022-01-03,600.00,2022-01-03,764.28,2022-01-03,442.68,2022-01-03,636.28
3,2022-01-04,210.80926,2022-01-04,401.21,2022-01-04,401.21,2022-01-04,401.21,2022-01-04,401.21,2022-01-04,401.21,2022-01-04,525.00,2022-01-04,476.20,2022-01-04,764.71
4,2022-01-05,154.48927,2022-01-05,530.00,2022-01-05,530.00,2022-01-05,530.00,2022-01-05,530.00,2022-01-05,530.00,2022-01-05,530.00,2022-01-05,163.19,2022-01-05,323.39


In [338]:
single_cycle_2hr_dict = {
    'NORD': (two_hour_df.NORD - two_hour_df.PUN).sum(),
    'CNOR': (two_hour_df.CNOR - two_hour_df.PUN).sum(),
    'CSUD': (two_hour_df.CSUD - two_hour_df.PUN).sum(),
    'SUD': (two_hour_df.SUD - two_hour_df.PUN).sum(),
    'SICI': (two_hour_df.SICI - two_hour_df.PUN).sum(),
    'SARD': (two_hour_df.SARD - two_hour_df.PUN).sum(),
}

single_cycle_2hr_gb = (two_hour_df.gb_max - two_hour_df.gb_min).sum()
print(f'GB revenue: {single_cycle_2hr_gb}')
single_cycle_2hr_df = pd.DataFrame(single_cycle_2hr_dict, index=['revenue']).transpose().round(2)
single_cycle_2hr_df

GB revenue: 93160.02


,revenue
NORD,112523.57
CNOR,112574.31
CSUD,109157.77
SUD,107685.38
SICI,108978.34
SARD,108292.71


In [258]:
fig = px.bar(
    single_cycle_2hr_df,
    color=single_cycle_2hr_df.revenue.values,
    color_continuous_scale='solar',
    color_continuous_midpoint=111e3,
    text_auto='.3s'
)

fig.update_layout(
    title='Single cycle 2 hour revenue',
    xaxis={'categoryorder':'total descending'},
    xaxis_title='',
    yaxis_title='€/MW',
    yaxis_range=[105e3, 113e3],
    template='ggplot2',
    width=1200,
    height=500,
)
fig.update_coloraxes(showscale=False)
fig.update_traces(textfont_size=14, textposition='inside')

fig.write_image('../figures/single-cycle-2hr.jpeg', scale=5, engine='orca')
fig

### Dual cycle 1 hr

In [259]:
price_df['am_pm'] = price_df.hour.apply(lambda x: 'am' if x<= 12 else 'pm')
price_df_am_min = price_df.loc[price_df.am_pm == 'am'].groupby('date').min()
price_df_am_max = price_df.loc[price_df.am_pm == 'am'].groupby('date').max()
price_df_pm_min = price_df.loc[price_df.am_pm == 'pm'].groupby('date').min()
price_df_pm_max = price_df.loc[price_df.am_pm == 'pm'].groupby('date').max()
price_df_am_min.head()

,datetime,hour,gb_price_eur,PUN,NORD,CNOR,CSUD,SUD,SARD,SICI,am_pm
date,,,,,,,,,,,
2022-01-01,2022-01-01 00:00:00,1,-22.85,67.99,67.99,67.99,67.99,67.99,67.99,67.99,am
2022-01-02,2022-01-02 00:00:00,1,137.49,100.00,100.00,100.00,100.00,100.00,100.00,100.00,am
2022-01-03,2022-01-03 00:00:00,1,219.61,83.27,83.27,83.27,83.27,83.27,83.27,83.27,am
2022-01-04,2022-01-04 00:00:00,1,238.88,105.00,105.00,105.00,105.00,105.00,105.00,105.00,am
2022-01-05,2022-01-05 00:00:00,1,90.10,75.38,75.38,75.38,75.38,75.38,75.38,75.38,am


In [340]:
dual_cycle_1hr_dict = {
    'NORD': (price_df_am_max.NORD - price_df_am_min.PUN).sum() + (price_df_pm_max.NORD - price_df_pm_min.PUN).sum(),
    'CNOR': (price_df_am_max.CNOR - price_df_am_min.PUN).sum() + (price_df_pm_max.CNOR - price_df_pm_min.PUN).sum(),
    'CSUD': (price_df_am_max.CSUD - price_df_am_min.PUN).sum() + (price_df_pm_max.CSUD - price_df_pm_min.PUN).sum(),
    'SUD': (price_df_am_max.SUD - price_df_am_min.PUN).sum() + (price_df_pm_max.SUD - price_df_pm_min.PUN).sum(),
    'SICI': (price_df_am_max.SICI - price_df_am_min.PUN).sum() + (price_df_pm_max.SICI - price_df_pm_min.PUN).sum(),
    'SARD': (price_df_am_max.SARD - price_df_am_min.PUN).sum() + (price_df_pm_max.SARD - price_df_pm_min.PUN).sum(),
}

dual_cycle_1hr_gb = (price_df_am_max.gb_price_eur - price_df_am_min.gb_price_eur).sum() + (price_df_pm_max.gb_price_eur - price_df_pm_min.gb_price_eur).sum()
print(f'GB revenue: {dual_cycle_1hr_gb}')
dual_cycle_1hr_df = pd.DataFrame(dual_cycle_1hr_dict, index=['revenue']).transpose().round(2)
dual_cycle_1hr_df

GB revenue: 59433.55


,revenue
NORD,89550.08
CNOR,89582.56
CSUD,84005.95
SUD,82104.61
SICI,81879.03
SARD,82092.70


In [261]:
fig = px.bar(
    dual_cycle_1hr_df,
    color=dual_cycle_1hr_df.revenue.values,
    color_continuous_scale='solar',
    color_continuous_midpoint=87e3,
    text_auto='.3s'
)

fig.update_layout(
    title='Dual cycle 1 hour revenue',
    xaxis={'categoryorder':'total descending'},
    xaxis_title='',
    yaxis_title='€/MW',
    yaxis_range=[80e3, 90e3],
    template='ggplot2',
    width=1200,
    height=500
)
fig.update_coloraxes(showscale=False)
fig.update_traces(textfont_size=14, textposition='inside')

fig.write_image('../figures/dual-cycle-1hr.jpeg', scale=5, engine='orca')
fig

### Dual cycle 2 hr

In [262]:
pun_min_am = price_df.loc[price_df.am_pm == 'am'].groupby('date')['PUN'].nsmallest(2).reset_index()[['date', 'PUN']].groupby('date').sum().reset_index()
nord_max_am = price_df.loc[price_df.am_pm == 'am'].groupby('date')['NORD'].nlargest(2).reset_index()[['date', 'NORD']].groupby('date').sum().reset_index()
cnor_max_am = price_df.loc[price_df.am_pm == 'am'].groupby('date')['CNOR'].nlargest(2).reset_index()[['date', 'CNOR']].groupby('date').sum().reset_index()
csud_max_am = price_df.loc[price_df.am_pm == 'am'].groupby('date')['CSUD'].nlargest(2).reset_index()[['date', 'CSUD']].groupby('date').sum().reset_index()
sud_max_am = price_df.loc[price_df.am_pm == 'am'].groupby('date')['SUD'].nlargest(2).reset_index()[['date', 'SUD']].groupby('date').sum().reset_index()
sard_max_am = price_df.loc[price_df.am_pm == 'am'].groupby('date')['SARD'].nlargest(2).reset_index()[['date', 'SARD']].groupby('date').sum().reset_index()
sici_max_am = price_df.loc[price_df.am_pm == 'am'].groupby('date')['SICI'].nlargest(2).reset_index()[['date', 'SICI']].groupby('date').sum().reset_index()
gb_min_am = price_df.loc[price_df.am_pm == 'am'].groupby('date')['gb_price_eur'].nsmallest(2).reset_index()[['date', 'gb_price_eur']].groupby('date').sum().reset_index().rename({'gb_price_eur': 'gb_min'}, axis=1)
gb_max_am = price_df.loc[price_df.am_pm == 'am'].groupby('date')['gb_price_eur'].nlargest(2).reset_index()[['date', 'gb_price_eur']].groupby('date').sum().reset_index().rename({'gb_price_eur': 'gb_max'}, axis=1)

two_hour_df_am = pd.concat([pun_min_am, nord_max_am, cnor_max_am, csud_max_am, sud_max_am, sard_max_am, sici_max_am, gb_min_am, gb_max_am], axis=1)
two_hour_df_am.head()

,date,PUN,date,NORD,date,CNOR,date,CSUD,date,SUD,date,SARD,date,SICI,date,gb_min,date,gb_max
0,2022-01-01,158.99000,2022-01-01,326.00000,2022-01-01,326.00000,2022-01-01,326.00000,2022-01-01,326.00000,2022-01-01,326.00000,2022-01-01,326.00000,2022-01-01,-33.08,2022-01-01,197.76
1,2022-01-02,240.60000,2022-01-02,343.06000,2022-01-02,343.06000,2022-01-02,343.06000,2022-01-02,343.06000,2022-01-02,343.06000,2022-01-02,343.06000,2022-01-02,296.94,2022-01-02,446.21
2,2022-01-03,167.77000,2022-01-03,471.51000,2022-01-03,471.51000,2022-01-03,471.51000,2022-01-03,471.51000,2022-01-03,471.51000,2022-01-03,471.51000,2022-01-03,442.68,2022-01-03,483.12
3,2022-01-04,210.80926,2022-01-04,392.80000,2022-01-04,392.80000,2022-01-04,392.80000,2022-01-04,392.80000,2022-01-04,392.80000,2022-01-04,392.80000,2022-01-04,479.56,2022-01-04,570.53
4,2022-01-05,154.48927,2022-01-05,450.96677,2022-01-05,450.96677,2022-01-05,450.96677,2022-01-05,450.96677,2022-01-05,450.96677,2022-01-05,450.96677,2022-01-05,184.46,2022-01-05,254.01


In [263]:
pun_min_pm = price_df.loc[price_df.am_pm == 'pm'].groupby('date')['PUN'].nsmallest(2).reset_index()[['date', 'PUN']].groupby('date').sum().reset_index()
nord_max_pm = price_df.loc[price_df.am_pm == 'pm'].groupby('date')['NORD'].nlargest(2).reset_index()[['date', 'NORD']].groupby('date').sum().reset_index()
cnor_max_pm = price_df.loc[price_df.am_pm == 'pm'].groupby('date')['CNOR'].nlargest(2).reset_index()[['date', 'CNOR']].groupby('date').sum().reset_index()
csud_max_pm = price_df.loc[price_df.am_pm == 'pm'].groupby('date')['CSUD'].nlargest(2).reset_index()[['date', 'CSUD']].groupby('date').sum().reset_index()
sud_max_pm = price_df.loc[price_df.am_pm == 'pm'].groupby('date')['SUD'].nlargest(2).reset_index()[['date', 'SUD']].groupby('date').sum().reset_index()
sard_max_pm = price_df.loc[price_df.am_pm == 'pm'].groupby('date')['SARD'].nlargest(2).reset_index()[['date', 'SARD']].groupby('date').sum().reset_index()
sici_max_pm = price_df.loc[price_df.am_pm == 'pm'].groupby('date')['SICI'].nlargest(2).reset_index()[['date', 'SICI']].groupby('date').sum().reset_index()
gb_min_pm = price_df.loc[price_df.am_pm == 'pm'].groupby('date')['gb_price_eur'].nsmallest(2).reset_index()[['date', 'gb_price_eur']].groupby('date').sum().reset_index().rename({'gb_price_eur': 'gb_min'}, axis=1)
gb_max_pm = price_df.loc[price_df.am_pm == 'pm'].groupby('date')['gb_price_eur'].nlargest(2).reset_index()[['date', 'gb_price_eur']].groupby('date').sum().reset_index().rename({'gb_price_eur': 'gb_max'}, axis=1)

two_hour_df_pm = pd.concat([pun_min_pm, nord_max_pm, cnor_max_pm, csud_max_pm, sud_max_pm, sard_max_pm, sici_max_pm, gb_min_pm, gb_max_pm], axis=1)
two_hour_df_pm.head()

,date,PUN,date,NORD,date,CNOR,date,CSUD,date,SUD,date,SARD,date,SICI,date,gb_min,date,gb_max
0,2022-01-01,247.09,2022-01-01,429.32,2022-01-01,429.32,2022-01-01,429.32,2022-01-01,429.32,2022-01-01,429.32,2022-01-01,429.32,2022-01-01,-49.38,2022-01-01,190.14
1,2022-01-02,299.23,2022-01-02,470.19,2022-01-02,470.19,2022-01-02,470.19,2022-01-02,470.19,2022-01-02,470.19,2022-01-02,485.00,2022-01-02,413.73,2022-01-02,499.76
2,2022-01-03,331.01,2022-01-03,600.00,2022-01-03,600.00,2022-01-03,600.00,2022-01-03,600.00,2022-01-03,600.00,2022-01-03,764.28,2022-01-03,463.20,2022-01-03,636.28
3,2022-01-04,302.40,2022-01-04,398.40,2022-01-04,398.40,2022-01-04,398.40,2022-01-04,398.40,2022-01-04,398.40,2022-01-04,525.00,2022-01-04,501.01,2022-01-04,764.71
4,2022-01-05,405.00,2022-01-05,530.00,2022-01-05,530.00,2022-01-05,530.00,2022-01-05,530.00,2022-01-05,530.00,2022-01-05,530.00,2022-01-05,177.21,2022-01-05,323.39


In [341]:
dual_cycle_2hr_dict = {
    'NORD': (two_hour_df_am.NORD - two_hour_df_am.PUN).sum() + (two_hour_df_pm.NORD - two_hour_df_pm.PUN).sum(),
    'CNOR': (two_hour_df_am.CNOR - two_hour_df_am.PUN).sum() + (two_hour_df_pm.CNOR - two_hour_df_pm.PUN).sum(),
    'CSUD': (two_hour_df_am.CSUD - two_hour_df_am.PUN).sum() + (two_hour_df_pm.CSUD - two_hour_df_pm.PUN).sum(),
    'SUD': (two_hour_df_am.SUD - two_hour_df_am.PUN).sum() + (two_hour_df_pm.SUD - two_hour_df_pm.PUN).sum(),
    'SICI': (two_hour_df_am.SICI - two_hour_df_am.PUN).sum() + (two_hour_df_pm.SICI - two_hour_df_pm.PUN).sum(),
    'SARD': (two_hour_df_am.SARD - two_hour_df_am.PUN).sum() + (two_hour_df_pm.SARD - two_hour_df_pm.PUN).sum(),
}

dual_cycle_2hr_gb = (two_hour_df_am.gb_max - two_hour_df_am.gb_min).sum() + (two_hour_df_pm.gb_max - two_hour_df_pm.gb_min).sum()
print(f'GB revenue: {dual_cycle_2hr_gb}')
dual_cycle_2hr_df = pd.DataFrame(dual_cycle_2hr_dict, index=['revenue']).transpose().round(2)
dual_cycle_2hr_df

GB revenue: 100828.54999999999


,revenue
NORD,163428.68
CNOR,163499.45
CSUD,152258.64
SUD,148370.07
SICI,147961.78
SARD,148016.40


In [265]:
fig = px.bar(
    dual_cycle_2hr_df,
    color=dual_cycle_2hr_df.revenue.values,
    color_continuous_scale='solar',
    color_continuous_midpoint=158e3,
    text_auto='.3s'
)

fig.update_layout(
    title='Dual cycle 2 hour revenue',
    xaxis={'categoryorder':'total descending'},
    xaxis_title='',
    yaxis_title='€/MW',
    yaxis_range=[145e3, 165e3],
    template='ggplot2',
    width=1200,
    height=500
)
fig.update_coloraxes(showscale=False)
fig.update_traces(textfont_size=14, textposition='inside')

fig.write_image('../figures/dual-cycle-2hr.jpeg', scale=5, engine='orca')
fig

### Summary plot

In [360]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=single_cycle_1hr_df.index,
    y=single_cycle_1hr_df.revenue.values,
    name='1 hour 1 cycle'
))
fig.add_trace(go.Bar(
    x=dual_cycle_1hr_df.index,
    y=dual_cycle_1hr_df.revenue.values,
    name='1 hour 2 cycle'
))
fig.add_trace(go.Bar(
    x=single_cycle_2hr_df.index,
    y=single_cycle_2hr_df.revenue.values,
    name='2 hour 1 cycle'
))
fig.add_trace(go.Bar(
    x=dual_cycle_2hr_df.index,
    y=dual_cycle_2hr_df.revenue.values,
    name='2 hour 2 cycle'
))

fig.update_traces(texttemplate='%{value: .4r}')
fig.update_layout(
    title='Italian MGP potential revenue',
    yaxis_title='€/MW',
    barmode='group',
    template='ggplot2',
    width=1200,
    height=500
    )

fig.write_image('../figures/revenue-summarised.jpeg', scale=5, engine='orca')
fig